[NB 03](https://github.com/radekosmulski/personalized_fashion_recs/blob/messing_around/03_Basic_lgbm_with_idxs_restart.ipynb) didn't quite work.

The two likely reasons for that are:
 - issues with how I am generating train data
 - false assumption that you can toss whatever at a ranking model and it will do the rest

In this notebook, I want to put the groundwork needed for growing a good solution. First of all, that will require a robust and fast local validation scheme. We know that using the last week of train data for validation work and tracks the leaderboard nicely.

Secondly, we want to start from a kernel of a solution that we can extend. This [notebook](https://www.kaggle.com/hengzheng/time-is-our-best-friend-v2) on kaggle seems to me like a great starting point.

The plan is to develop the functionality needed for a nice setup of a solution that we will reuse in NB 05. Along the way I hope to learn a bit more about the data, about some of the trends that I might want to model through the features I will engineer.

The plan is:
* implement a quick training pipeline leading to good validation
* train a ranking model on candidates we know to be good
* only generate new training data / candidates while validating whether we are moving in the right direction using local CV
* start with building sensible features and see whether they move the needle on the score

The truth is I do not know what will work. These RecSys models are a completely new breed of models to me. But I can set the problem up in a way as to help me learn. And that is what I am going to do :).

Once I get this working I will breathe a sigh of relief and will jump into reading papers and drawing inspiration from there.

Let's get started.

NOTE: You are welcome to check out the earlier code that I wrote which can be found on [this branch](https://github.com/radekosmulski/personalized_fashion_recs/tree/messing_around). I learned a lot about RecSys models and this particular problem through it. But it has quite a few bugs and a couple of issues I now know off with regards to the approach. **You should not need any of the earlier code to run the notebooks in the main branch of this repo.**

In [16]:
!wget https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py

dyld[12938]: Library not loaded: /opt/homebrew/opt/libunistring/lib/libunistring.2.dylib
  Referenced from: <DC03D607-E12F-36EF-B99B-F9FAE69BDA4E> /opt/homebrew/Cellar/libidn2/2.3.4/lib/libidn2.0.dylib
  Reason: tried: '/opt/homebrew/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/opt/homebrew/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/usr/local/lib/libunistring.2.dylib' (no such file), '/usr/lib/libunistring.2.dylib' (no such file, not in dyld cache), '/opt/homebrew/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/opt/homebrew/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/usr/local/lib/libunistring.2.dylib' (no such file), '/usr/lib/libunistring.2.dylib' (no such file, not in dyld cache)


In [17]:
# helper functions
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from average_precision import apk

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)

We want this to be fast. I can get as much RAM as I will ever need through VMs on GCP, but that is not the point. I want to see how far I can push my local hardware, but this goes even beyond that.

I need the speed to make a good use of my time as I continue to build my understanding of what RecSys models are about. And the path to this leads through making the data I will work on smaller.

In [18]:
%%time
import pandas as pd

transactions = pd.read_csv('transactions_train.csv', dtype={"article_id": "str"})
customers = pd.read_csv('customers.csv')
articles = pd.read_csv('articles.csv', dtype={"article_id": "str"})

CPU times: user 19.1 s, sys: 1.66 s, total: 20.7 s
Wall time: 20.8 s


In [19]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id         3846387204
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [20]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 8.0 GB


In [21]:
%%time
transactions['customer_id'].nunique()

CPU times: user 4.5 s, sys: 143 ms, total: 4.64 s
Wall time: 4.64 s


1362281

In [22]:
%%time
transactions['customer_id'] = customer_hex_id_to_int(transactions['customer_id'])
transactions['customer_id'].nunique()

CPU times: user 13.5 s, sys: 1.51 s, total: 15 s
Wall time: 15.4 s


1362281

In [23]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id          254306592
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [24]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       uint64 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 4.7 GB


Nice!

Initially, I wanted to get rid of the `t_dat` column but on second thought I am not a fan.

I am all for speed and reducing weight, but the main purpose of this activity is to increase developer productivity.

If I fall back down to ints representing year, week, day I will be certainly trading developer productivity for fewer CPU cycles that are needed (and I want to go in the exact opposite direction! developer productivity > (nearly) anything else)

In [25]:
%%time

transactions.t_dat = pd.to_datetime(transactions.t_dat, format='%Y-%m-%d')

CPU times: user 1.67 s, sys: 129 ms, total: 1.8 s
Wall time: 1.8 s


In [26]:
transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

In [27]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
 5   week              int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), uint64(1)
memory usage: 3.2 GB


Let's do something about the `article_id` (both here and on `articles`) and let's take a closer look at `price`, `sales_channel_id` and `week`.

In [28]:
transactions.article_id = article_id_str_to_int(transactions.article_id)
articles.article_id = article_id_str_to_int(articles.article_id)

transactions.week = transactions.week.astype('int8')
transactions.sales_channel_id = transactions.sales_channel_id.astype('int8')
transactions.price = transactions.price.astype('float32')

In [29]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        int32         
 3   price             float32       
 4   sales_channel_id  int8          
 5   week              int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 788.2 MB


In [30]:
transactions.drop(columns='t_dat').info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   customer_id       uint64 
 1   article_id        int32  
 2   price             float32
 3   sales_channel_id  int8   
 4   week              int8   
dtypes: float32(1), int32(1), int8(2), uint64(1)
memory usage: 545.7 MB


Well, this is interesting. There are very few unique `t_dat` values hence despite it being a scary `datetime64` it takes up very little memory!

Keeping it for convenience is definitely the way to go.

Let's take a brief look at the `customers` and `articles` dfs.

In [31]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 512.3 MB


In [32]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int32 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

Well, this stuff will be getting merged with our transactions df at some point, so I guess we can also make this smaller and easier to work with down the road.

In [33]:
customers['club_member_status'].unique()

array(['ACTIVE', nan, 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [34]:
customers.customer_id = customer_hex_id_to_int(customers.customer_id)
for col in ['FN', 'Active', 'age']:
    customers[col].fillna(-1, inplace=True)
    customers[col] = customers[col].astype('int8')

In [35]:
customers.club_member_status = Categorize().fit_transform(customers[['club_member_status']]).club_member_status
customers.postal_code = Categorize().fit_transform(customers[['postal_code']]).postal_code
customers.fashion_news_frequency = Categorize().fit_transform(customers[['fashion_news_frequency']]).fashion_news_frequency

In [36]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   customer_id             1371980 non-null  uint64
 1   FN                      1371980 non-null  int8  
 2   Active                  1371980 non-null  int8  
 3   club_member_status      1371980 non-null  int8  
 4   fashion_news_frequency  1371980 non-null  int8  
 5   age                     1371980 non-null  int8  
 6   postal_code             1371980 non-null  int32 
dtypes: int32(1), int8(5), uint64(1)
memory usage: 22.2 MB


In [37]:
for col in articles.columns:
    if articles[col].dtype == 'object':
        articles[col] = Categorize().fit_transform(articles[[col]])[col]

In [38]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   article_id                    105542 non-null  int32
 1   product_code                  105542 non-null  int64
 2   prod_name                     105542 non-null  int32
 3   product_type_no               105542 non-null  int64
 4   product_type_name             105542 non-null  int16
 5   product_group_name            105542 non-null  int8 
 6   graphical_appearance_no       105542 non-null  int64
 7   graphical_appearance_name     105542 non-null  int8 
 8   colour_group_code             105542 non-null  int64
 9   colour_group_name             105542 non-null  int8 
 10  perceived_colour_value_id     105542 non-null  int64
 11  perceived_colour_value_name   105542 non-null  int8 
 12  perceived_colour_master_id    105542 non-null  int64
 13  perceived_colo

In [39]:
for col in articles.columns:
    if articles[col].dtype == 'int64':
        articles[col] = articles[col].astype('int32')

And this concludes our raw data preparation step! Let's now write everything back to disk.

In [40]:
transactions.sort_values(['t_dat', 'customer_id'], inplace=True)

In [41]:
%%time

transactions.to_parquet('transactions_train.parquet')
customers.to_parquet('customers.parquet')
articles.to_parquet('articles.parquet')

CPU times: user 3.28 s, sys: 182 ms, total: 3.46 s
Wall time: 3.59 s
